In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
data = pd.read_csv('data/combined_data_cleaned.csv')

In [3]:
data.head()

,date,state,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations_per_million,share_doses_used,fips,deaths,new_cases,total_cases,population
0,2020-01-21,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,0,1,1,7614800
1,2020-01-22,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,0,0,1,7614800
2,2020-01-23,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,0,0,1,7614800
3,2020-01-24,Illinois,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,0,1,1,12672400
4,2020-01-24,Washington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,0,0,1,7614800


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26359 entries, 0 to 26358
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date                                 26359 non-null  object 
 1   state                                26359 non-null  object 
 2   total_vaccinations                   9020 non-null   float64
 3   total_distributed                    9020 non-null   float64
 4   people_vaccinated                    9020 non-null   float64
 5   people_fully_vaccinated_per_hundred  9020 non-null   float64
 6   total_vaccinations_per_hundred       9020 non-null   float64
 7   people_fully_vaccinated              9020 non-null   float64
 8   people_vaccinated_per_hundred        9020 non-null   float64
 9   distributed_per_hundred              9020 non-null   float64
 10  daily_vaccinations_raw               9020 non-null   float64
 11  daily_vaccinations_per_milli

In [5]:
#create new feature that lists the number of new cases based on total population

data['new_cases_per_hundred'] = data['new_cases'] / data['population'] * 100

In [6]:
#drop all columns that are not relevant to the model

model_data = data[['date', 'state', 'people_vaccinated_per_hundred', 'new_cases_per_hundred']]

In [7]:
model_data.head()

,date,state,people_vaccinated_per_hundred,new_cases_per_hundred
0,2020-01-21,Washington,NaN,0.000013
1,2020-01-22,Washington,NaN,0.000000
2,2020-01-23,Washington,NaN,0.000000
3,2020-01-24,Illinois,NaN,0.000008
4,2020-01-24,Washington,NaN,0.000000


In [8]:
#number of total unique dates

model_data['date'].nunique()

521

In [9]:
#number of unique dates since vaccinations

model_data[model_data['people_vaccinated_per_hundred'].notnull()]['date'].nunique()

164

In [10]:
#first date with vaccination data is 2021-01-12

model_data.date = pd.to_datetime(model_data.date)
first = model_data[model_data['people_vaccinated_per_hundred'].notnull()]['date'].take([0])

C:\Users\USFK04487\Anaconda3\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [11]:
#add first 100 days of vaccination data to training set, rest to testing set

first + timedelta(days=100)

17339   2021-04-22
Name: date, dtype: datetime64[ns]

In [12]:
train = model_data.loc[model_data['date']<='2021-04-22']
test = model_data.loc[model_data['date']>'2021-04-22']

In [13]:
#split into X and y

y_train = train.new_cases_per_hundred
y_test = test.new_cases_per_hundred

X_train = train.drop('new_cases_per_hundred', axis=1)
X_test = test.drop('new_cases_per_hundred', axis=1)